In [68]:
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import pandas as pd
from random import sample

def make_wordmap(z,y,n):
    plt.scatter(z, y)
    for i, txt in enumerate(n):
        plt.annotate(txt, (z[i], y[i]))
        
  
 
dummy = """This is a sample sentence, 
                  showing off the stop words filtration."""


example_sent = '''During the first part of your life,
                  you only become aware of happiness once you have lost it.
                  Then an age comes, a second one, in which you already know, 
                  at the moment when you begin to experience true happiness,
                  that you are, at the end of the day, going to lose it. When
                  I met Belle, I understood that I had just entered this second
                  age. I also understood that I hadn’t reached the third age, in
                  which anticipation of the loss of happiness prevents you from living.'''

stop_words = set(stopwords.words('english'))


no_punct = example_sent.translate(str.maketrans('', '', string.punctuation))
word_tokens = word_tokenize(no_punct)
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

window_size = 2

n_samplings = 2

def n_neighbors(lst, n):
    result = []
    for i in range(len(lst)):
        left = max(0, i - n)
        right = min(len(lst), i + n + 1)
        neighbors = lst[left:right]
        neighbors.remove(lst[i])
        result.append((lst[i],neighbors))
    return result



context = n_neighbors(filtered_sentence, window_size)



def make_context(lst,negative=False):
    result = []
    for x,y in lst:
         for word in y:
            if(not negative):
                result.append((x,word,1))
            else:
                result.append((x,word,0))
    return result


def make_n_sampling(lst, n):
    result = []
    for i in range(len(lst)):
        lower = max(0, i-n)
        upper = min(len(lst)-1, i+n)
        sublist = [lst[j] for j in range(len(lst)) if j < lower or j > upper]
        result.append((lst[i],sublist))
    return result

context = n_neighbors(filtered_sentence, window_size)
n_context = make_n_sampling(filtered_sentence, window_size) 

df_sampling = pd.DataFrame(make_context(context), columns =['Main', 'Context', 'Label'])

n_context_list = make_context(n_context, negative=True)



words_with_n_sampling = set([x[0] for x in n_context_list])

neg_samp_lst = []
for word in words_with_n_sampling:
    tuples_filtered = [tup for tup in n_context_list if tup[0] == word]
    rand_sample = sample(tuples_filtered, min(len(tuples_filtered),n_samplings))
    neg_samp_lst += rand_sample
    
df_n_sampling = pd.DataFrame(data = neg_samp_lst, columns =['Main', 'Context', 'Label'])


final_df = pd.concat([df_n_sampling,df_sampling]).sample(frac=1).reset_index(drop=True)
final_df





,Main,Context,Label
0,already,begin,0
1,happiness,experience,1
2,understood,’,1
3,life,first,1
4,become,know,0
...,...,...,...
211,reached,second,0
212,’,also,1
213,lose,going,1
214,moment,loss,0
